In [4]:
import QuantLib as ql

In [51]:
class vanillaPrice:
    calculation_date = ql.Date(31, 12, 2018)
    risk_free_rate = 0
    libor_rate = 0

    def __init__(self, risk_free_rate, libor_rate):
        self.risk_free_rate = risk_free_rate
        self.libor_rate = libor_rate

    def discountCurve(self):
        day_count = ql.Actual365Fixed()
        discount_curve = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.risk_free_rate, day_count))
        return discount_curve

    def liborIndex(self):
        day_count = ql.Actual365Fixed()
        libor_curve = ql.YieldTermStructureHandle(ql.FlatForward(self.calculation_date, self.libor_rate, day_count))
        libor_index = ql.USDLibor(ql.Period(3, ql.Months), libor_curve)
        return libor_index

    def fixFloatLegSchedule(self):
        calendar = ql.UnitedStates()
        settle_date = calendar.advance(self.calculation_date, 5, ql.Days)
        maturity_date = calendar.advance(settle_date, 10, ql.Years)
        fixed_leg_tenor = ql.Period(6, ql.Months)
        fixed_schedule = ql.Schedule(settle_date, maturity_date, 
                                    fixed_leg_tenor, calendar,
                                    ql.ModifiedFollowing, ql.ModifiedFollowing,
                                    ql.DateGeneration.Forward, False)
        float_leg_tenor = ql.Period(3, ql.Months)
        float_schedule = ql.Schedule (settle_date, maturity_date, 
                                    float_leg_tenor, calendar,
                                    ql.ModifiedFollowing, ql.ModifiedFollowing,
                                    ql.DateGeneration.Forward, False)
        return fixed_schedule, float_schedule

    def price(self, notional, fixed_rate, float_spread):
        fixed_leg_daycount = ql.Actual360()
        float_leg_daycount = ql.Actual360()
        discount_curve = self.discountCurve()
        libor_index = self.liborIndex()
        fixed_schedule, float_schedule = self.fixFloatLegSchedule()
        ir_swap = ql.VanillaSwap(ql.VanillaSwap.Payer, notional, fixed_schedule, 
                    fixed_rate, fixed_leg_daycount, float_schedule,
                    libor_index, float_spread, float_leg_daycount )
        swap_engine = ql.DiscountingSwapEngine(discount_curve)
        ir_swap.setPricingEngine(swap_engine)
        return ir_swap

In [52]:
vanilla = vanillaPrice(0.01, 0.02)
price = vanilla.price(10000000, 0.025, 0.004)

In [55]:
for i, cf in enumerate(price.leg(0)):
    print(i+1, cf.date(), cf.amount())

1 July 8th, 2019 125694.4444444441
2 January 8th, 2020 127777.77777777777
3 July 8th, 2020 126388.88888888866
4 January 8th, 2021 127777.77777777777
5 July 8th, 2021 125694.4444444441
6 January 10th, 2022 129166.66666666688
7 July 8th, 2022 124305.55555555501
8 January 9th, 2023 128472.22222222232
9 July 10th, 2023 126388.88888888866
10 January 8th, 2024 126388.88888888866
11 July 8th, 2024 126388.88888888866
12 January 8th, 2025 127777.77777777777
13 July 8th, 2025 125694.4444444441
14 January 8th, 2026 127777.77777777777
15 July 8th, 2026 125694.4444444441
16 January 8th, 2027 127777.77777777777
17 July 8th, 2027 125694.4444444441
18 January 10th, 2028 129166.66666666688
19 July 10th, 2028 126388.88888888866
20 January 8th, 2029 126388.88888888866


In [56]:
for i, cf in enumerate(price.leg(1)):
    print(i+1, cf.date(), cf.amount())

RuntimeError: Missing USDLibor3M Actual/360 fixing for January 4th, 2019